# <center> WillGraceApartment </center>

![piantina](./img/piantinaGirata.png)
![piantina](./img/Screenshot_1.png)
![piantina](./img/Screenshot_2.png)
![piantina](./img/Screenshot_3.png)

In [1]:
from pyplasm import *
import csv
%run "../2016-11-25/workshop_06.ipynb"


Evaluating fenvs.py..
...fenvs.py imported in 0.00351992968918 seconds


In [2]:
"""constant var and path"""
# Path
pathExternalWalls = "./lines/external.lines"
pathInternalWalls = "./lines/internal.lines"
pathGrindWalls = "./lines/grind.lines"
pathFloor = "./lines/floor.lines"
pathHoles = "./lines/holes.lines"
pathBalconyFloor = "./lines/balconyFloor.lines"
pathDoors = "./lines/doors2.lines"
pathMainEntrance = "./lines/portoneIngresso.lines"
pathWindows = "./lines/windows3.lines"
pathAperture = "./lines/aperture.lines"

# Constant var
externalWallsThickness = 5
internalWallsThickness = 2
grindWallsThickness = 1
grindHeight = 10
houseHeight = 40
windowsHeight = 20
windowsHeightFromFloor = 10
doorsHeight = 25

# Material Orange
#ambientRGBA = [1,0,0,1]
#diffuseRGBA = [0,1,0,1]
#specularRGBA = [0,0,1,0]
#emissionRGBA = [1,0,0,1]
#shininess = 100

wx = [0.6, 8.2, 0.6, 8.2, 0.6, 8.2, 0.6]
wy = 0.3
wz = [0.6, 4.25, 0.6, 4.25, 0.6, 4.25, 0.6, 4.25, 0.6]
wb = [[1,1,1,1,1,1,1],[1,0,0,0,0,0,1],[1,1,1,1,1,1,1],[1,0,1,0,1,0,1],[1,1,1,1,1,1,1],[1,0,1,0,1,0,1],[1,1,1,1,1,1,1],[1,0,1,0,1,0,1],[1,1,1,1,1,1,1]]

In [3]:
def readWindows(file_name, wx, wy, wz, wb):
    i = 0
    struct = []
    with open(file_name, 'rb') as csvfile:
        buildReader = csv.reader(csvfile)
        for row in buildReader:
            x1 = float(row[0])
            y1 = float(row[1])
            x2 = float(row[2])
            y2 = float(row[3])
            i += 2
            
            a = angle(x1, y1, x2, y2)
            window = ggpl_createWindow(wx, wy, wz, wb)
            if(a == 0):
                window = T([1,2,3])([x1, y1+externalWallsThickness/2.0, windowsHeightFromFloor])(window)
            if(a > 1.0 and a < 3.0):
                window = R([1,2])(a)(window)
                window = T([1,2,3])([x1+externalWallsThickness/2.0, y1, windowsHeightFromFloor])(window)
                #window = T(1)(wy)(window)
            if(a > 3.0):
                window = R([1,2])(a)(window)
                window = T([1,2,3])([x1, y1+externalWallsThickness/2.0, windowsHeightFromFloor])(window)
                #window = T(2)(wy)(window)
            struct.append(window)
    return STRUCT(struct)

In [4]:
def readWindowBox(file_name, wx, wy, wz, wb):
    i = 0
    struct = []
    with open(file_name, 'rb') as csvfile:
        buildReader = csv.reader(csvfile)
        for row in buildReader:
            x1 = float(row[0])
            y1 = float(row[1])
            x2 = float(row[2])
            y2 = float(row[3])
            i += 2
            
            a = angle(x1, y1, x2, y2)
            window = BOX([1,2,3])(ggpl_createWindow(wx, externalWallsThickness, wz, wb))
            window = R([1,2])(a)(window)
            window = T([1,2,3])([x1, y1, windowsHeightFromFloor])(window)
            if(a > 1.0 and a < 3.0):
                window = T(1)(externalWallsThickness)(window)
            if(a > 3.0):
                window = T(2)(externalWallsThickness)(window)
            struct.append(window)
    return STRUCT(struct)

In [5]:
def readDoorBox(file_name, dy, dz):
    i = 0
    struct = []
    with open(file_name, 'rb') as csvfile:
        buildReader = csv.reader(csvfile)
        for row in buildReader:
            x1 = float(row[0])
            y1 = float(row[1])
            x2 = float(row[2])
            y2 = float(row[3])
            i += 2
            dx = dist(x1, y1, x2, y2)
            a = angle(x1, y1, x2, y2)
            door = CUBOID([dx, dy, dz])
            door = R([1,2])(a)(door)
            door = T([1,2])([x1, y1])(door)
            if(a > 1.0 and a < 3.0):
                door = T(1)(externalWallsThickness)(door)
            if(a > 3.0):
                door = T(2)(externalWallsThickness)(door)
            struct.append(door)
    return STRUCT(struct)

In [8]:
"""This function uses the constants above and creates an HPC object of an house"""

def MyAparment():
    # HPC
    externalWalls = readFile(pathExternalWalls)
    internalWalls = readFile(pathInternalWalls)
    grind = readFile(pathGrindWalls)
    aperture = readFile(pathAperture)
    holes = readFile(pathHoles)
    ground = readFile(pathFloor)
    balconyFloor = readFile(pathBalconyFloor)
    
    # OFFSET
    externalWalls = OFFSET([externalWallsThickness,externalWallsThickness])(externalWalls)
    internalWalls = OFFSET([internalWallsThickness,internalWallsThickness])(internalWalls)
    grind = OFFSET([grindWallsThickness,grindWallsThickness])(grind)
    aperture = OFFSET([externalWallsThickness+1, externalWallsThickness+1])(aperture)
    
    # PROD
    externalWalls = PROD([externalWalls, QUOTE([houseHeight])])
    internalWalls = PROD([internalWalls, QUOTE([houseHeight])])
    grind = PROD([grind, QUOTE([grindHeight])])
    aperture = PROD([aperture, QUOTE([doorsHeight])])
    
    # BOX
    windowBoxs = readWindowBox(pathWindows, wx, externalWallsThickness, wz, wb)
    doorBoxs = readDoorBox(pathDoors, 5, doorsHeight)
    
    # OBJECT
    windows = readWindows(pathWindows, wx, 0.2, wz, wb)
    #doors = readDoor(pathWindows, wx, 0.2, wz, wb)
    holes = SOLIDIFY(holes)
    ground = SOLIDIFY(ground)
    
    # DIFF
    grind = DIFF([grind, externalWalls])
    internalWalls = DIFF([internalWalls, externalWalls])  
    internalWalls = DIFF([internalWalls, doorBoxs])
    externalWalls = DIFF([externalWalls, windowBoxs])
    externalWalls = DIFF([externalWalls, doorBoxs])
    externalWalls = DIFF([externalWalls, aperture])
    floor = DIFF([ground, holes])
    
    # COLOR AND TEXTURE
    grind = material(ambientRGBA, diffuseRGBA, specularRGBA, emissionRGBA, shininess, grind) #vetro
    externalWalls = COLOR(GREY)(externalWalls)    
    internalWalls = COLOR(GREEN)(internalWalls)
    ground = COLOR(WOOD)(ground)
    balconyFloor = COLOR(GREY)(SOLIDIFY(balconyFloor))
      
    # FINAL STRUCTURE
    struct = STRUCT([grind, windows, ground, balconyFloor, externalWalls, internalWalls])
    return struct


In [9]:
VIEW(MyAparment())

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x000000000ABAF750> >